In [ ]:
"""
Spending Analyzer Agent
Performs expense analysis and flags anomalies.
"""
import autogen
from typing import Dict, Any, List
from app.tools.financial_tools import FinancialTools

class SpendingAnalyzerAgent:
    """Expense analysis agent"""
    
    def __init__(self, llm_config: Dict[str, Any]):
        self.llm_config = llm_config
        self.tools = FinancialTools()
        
        self.agent = autogen.AssistantAgent(
            name="SpendingAnalyzer",
            system_message="""
You are the Spending Analyzer Agent — an expert in expense analytics.

Responsibilities:
1. Break down spending by category and department
2. Compare actuals vs. budget allocations
3. Detect unusual or anomalous transactions
4. Deliver actionable insights about spending patterns

Response guidelines:
- Present currency values and percentages clearly
- Highlight variances (over/under budget)
- Call out anomalies with severity
- Recommend savings or reallocation actions
- Use lightweight tables or bullet lists for clarity

Always provide detailed, decision-ready insights.
""",
            llm_config=llm_config,
            human_input_mode="NEVER",
            max_consecutive_auto_reply=5,
        )
    
    def analyze(self, transactions: List[Dict], budget: Dict) -> str:
        """
        Analyze spending and produce a narrative report.

        Args:
            transactions: List of transactions
            budget: Budget dictionary

        Returns:
            Analysis report
        """
        # Run analysis tool
        analysis = self.tools.analyze_spending(transactions, budget)
        
        # Detect anomalies
        anomalies = self.tools.detect_anomalies(transactions)
        
        prompt = f"""
Create a comprehensive spending analysis based on:

Summary:
- Total Budget: {analysis['summary']['total_budget']}
- Total Spending: {analysis['summary']['total_spending']}
- Remaining: {analysis['summary']['remaining']}
- Utilization: {analysis['summary']['utilization_pct']}%

Category Breakdown:
{self._format_category_breakdown(analysis['category_breakdown'])}

Anomalies detected: {len(anomalies)} transactions
{self._format_anomalies(anomalies)}

Requirements:
1. Provide an executive overview of spending health
2. Highlight categories that are over or under budget with reasons
3. Explain the anomalies and their potential impact
4. Recommend corrective actions or optimizations
"""
        
        result = self.agent.generate_reply([{
            "role": "user",
            "content": prompt
        }])
        
        return result.get('content', 'Unable to generate spending analysis')
    
    def _format_category_breakdown(self, breakdown: Dict) -> str:
        """Format category breakdown for prompt"""
        lines = []
        for category, data in breakdown.items():
            lines.append(
                f"- {category}: Actual={data['actual']}, Budget={data['budget']}, "
                f"Variance={data['variance']} ({data['variance_pct']}%)"
            )
        return '\n'.join(lines)
    
    def _format_anomalies(self, anomalies: List[Dict]) -> str:
        """Format anomalies for prompt"""
        if not anomalies:
            return "No anomalous transactions"
        
        lines = []
        for anomaly in anomalies[:5]:  # Limit to 5
            lines.append(
                f"- {anomaly['date']}: {anomaly['amount']} ({anomaly['category']}) - {anomaly['reason']}"
            )
        return '\n'.join(lines)